# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120486e+02     1.496448e+00
 * time: 0.09202003479003906
     1     1.109452e+01     9.943075e-01
 * time: 0.9985170364379883
     2    -1.233186e+01     1.084438e+00
 * time: 1.1586480140686035
     3    -3.442275e+01     7.661405e-01
 * time: 1.2974388599395752
     4    -4.826875e+01     6.156790e-01
 * time: 1.4582080841064453
     5    -5.733805e+01     1.993634e-01
 * time: 1.6111230850219727
     6    -6.011613e+01     9.010097e-02
 * time: 1.7275218963623047
     7    -6.104975e+01     6.195404e-02
 * time: 1.8425440788269043
     8    -6.142534e+01     6.919091e-02
 * time: 1.9543380737304688
     9    -6.171897e+01     3.057378e-02
 * time: 2.074384927749634
    10    -6.189914e+01     2.627182e-02
 * time: 2.1923940181732178
    11    -6.202209e+01     1.760750e-02
 * time: 2.2996459007263184
    12    -6.208024e+01     1.644722e-02
 * time: 2.4146058559417725
    13    -6.212763e+01     1.501717e-02
 * time: 2.5242810249

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671055
    AtomicLocal         -18.8557656
    AtomicNonlocal      14.8522643
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485360 
    Xc                  -19.3336816

    total               -62.261666452780
